<a href="https://colab.research.google.com/github/arijeetchoudhury100/Wavelet_CNN_keras/blob/main/wavelet_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.2-rc3

In [ ]:
import numpy as np
import math
import cv2
import pywt
import os
from PIL import Image
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model, Sequential
import seaborn as sb
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Activation, Dropout, Lambda, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.metrics import classification_report,confusion_matrix
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Layer

In [ ]:
K.set_image_data_format('channels_first')

In [46]:
print(pywt.wavelist(kind='discrete'))

['bior1.1', 'bior1.3', 'bior1.5', 'bior2.2', 'bior2.4', 'bior2.6', 'bior2.8', 'bior3.1', 'bior3.3', 'bior3.5', 'bior3.7', 'bior3.9', 'bior4.4', 'bior5.5', 'bior6.8', 'coif1', 'coif2', 'coif3', 'coif4', 'coif5', 'coif6', 'coif7', 'coif8', 'coif9', 'coif10', 'coif11', 'coif12', 'coif13', 'coif14', 'coif15', 'coif16', 'coif17', 'db1', 'db2', 'db3', 'db4', 'db5', 'db6', 'db7', 'db8', 'db9', 'db10', 'db11', 'db12', 'db13', 'db14', 'db15', 'db16', 'db17', 'db18', 'db19', 'db20', 'db21', 'db22', 'db23', 'db24', 'db25', 'db26', 'db27', 'db28', 'db29', 'db30', 'db31', 'db32', 'db33', 'db34', 'db35', 'db36', 'db37', 'db38', 'dmey', 'haar', 'rbio1.1', 'rbio1.3', 'rbio1.5', 'rbio2.2', 'rbio2.4', 'rbio2.6', 'rbio2.8', 'rbio3.1', 'rbio3.3', 'rbio3.5', 'rbio3.7', 'rbio3.9', 'rbio4.4', 'rbio5.5', 'rbio6.8', 'sym2', 'sym3', 'sym4', 'sym5', 'sym6', 'sym7', 'sym8', 'sym9', 'sym10', 'sym11', 'sym12', 'sym13', 'sym14', 'sym15', 'sym16', 'sym17', 'sym18', 'sym19', 'sym20']


In [ ]:
#define the wavelet
#see pywt.wavelist(kind='discrete') for available wavelets 
wavelet = pywt.Wavelet('haar')

class DWT_Pooling(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super(DWT_Pooling, self).__init__(**kwargs)
        
    def build(self, input_shape):
        super(DWT_Pooling, self).build(input_shape) 
    
    @tf.function
    def call(self, inputs):
        band_low = wavelet.rec_lo
        band_high = wavelet.rec_hi
        assert len(band_low) == len(band_high)
        band_length = len(band_low)
        assert band_length % 2 == 0
        band_length_half = math.floor(band_length / 2)

        input_height = inputs.shape[2]
        input_width = inputs.shape[3]

        L1 = input_height
        L = math.floor(L1 / 2)
        matrix_h = np.zeros( ( L,      L1 + band_length - 2 ), dtype=np.float32)
        matrix_g = np.zeros( ( L1 - L, L1 + band_length - 2 ), dtype=np.float32)
        end = None if band_length_half == 1 else (-band_length_half+1)
        
        index = 0
        for i in range(L):
            for j in range(band_length):
                matrix_h[i, index+j] = band_low[j]
            index += 2
        matrix_h_0 = matrix_h[0:(math.floor(input_height / 2)), 0:(input_height + band_length - 2)]
        matrix_h_1 = matrix_h[0:(math.floor(input_width / 2)), 0:(input_width + band_length - 2)]

        index = 0
        for i in range(L1 - L):
            for j in range(band_length):
                matrix_g[i, index+j] = band_high[j]
            index += 2

        matrix_g_0 = matrix_g[0:(input_height - math.floor(input_height / 2)),0:(input_height + band_length - 2)]
        matrix_g_1 = matrix_g[0:(input_width - math.floor(input_width / 2)),0:(input_width + band_length - 2)]

        matrix_h_0 = matrix_h_0[:,(band_length_half-1):end]
        matrix_h_1 = matrix_h_1[:,(band_length_half-1):end]
        matrix_h_1 = np.transpose(matrix_h_1)
        matrix_g_0 = matrix_g_0[:,(band_length_half-1):end]
        matrix_g_1 = matrix_g_1[:,(band_length_half-1):end]
        matrix_g_1 = np.transpose(matrix_g_1)

        matrix_low_0 = tf.convert_to_tensor(matrix_h_0,dtype=tf.float32)
        matrix_low_1 = tf.convert_to_tensor(matrix_h_1,dtype=tf.float32)
        matrix_high_0 = tf.convert_to_tensor(matrix_g_0,dtype=tf.float32)
        matrix_high_1 = tf.convert_to_tensor(matrix_g_1,dtype=tf.float32)
        
        L = tf.matmul(matrix_low_0, inputs)
        H = tf.matmul(matrix_high_0, inputs)
        LL = tf.matmul(L, matrix_low_1)
        LH = tf.matmul(L, matrix_high_1)
        HL = tf.matmul(H, matrix_low_1)
        HH = tf.matmul(H, matrix_high_1)
        return LL    
    
    def get_config(self):
        config = super(DWT_Pooling, self).get_config()
        return config

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], input_shape[2]//2, input_shape[3]//2)

In [ ]:
def create_model(input_shape=(1,28,28), num_classes = 1, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)

  inputs = Input(shape=input_shape)
  
  output = Conv2D(16,(3,3),padding='same',use_bias=False)(inputs)
  output = BatchNormalization(scale=False,center=True)(output)
  output = Activation('relu')(output)
  #output = MaxPooling2D()(output)
  output = DWT_Pooling()(output)

  output = Conv2D(32,(3,3),padding='same',use_bias=False)(output)
  output = BatchNormalization(scale=False,center=True)(output)
  output = Activation('relu')(output)
  #output = MaxPooling2D()(output)
  output = DWT_Pooling()(output)

  output = Flatten()(output)
  output = Dense(256,activation='relu')(output)
  output = Dropout(0.3)(output)
  if num_classes == 1:
    activation = 'sigmoid'
  else:
    activation = 'softmax'
  output = Dense(num_classes,activation=activation,bias_initializer=output_bias)(output)
  model = Model(inputs,output)
  return model

In [55]:
model = create_model(input_shape=(1,28,28),num_classes=10)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1, 28, 28)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 28, 28)        144       
_________________________________________________________________
batch_normalization_8 (Batch (None, 16, 28, 28)        84        
_________________________________________________________________
activation_8 (Activation)    (None, 16, 28, 28)        0         
_________________________________________________________________
dwt__pooling_7 (DWT_Pooling) (None, 16, 14, 14)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 14, 14)        4608      
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 14, 14)        42  

# MNIST example

In [47]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

In [48]:
x_train.shape

(60000, 28, 28)

In [49]:
x_train = x_train/255.
x_test = x_test/255.

In [50]:
from tensorflow.keras.utils import to_categorical
y_train_oh = to_categorical(y_train,10)
y_test_oh = to_categorical(y_test,10)

In [51]:
num_train = x_train.shape[0]
num_test = x_test.shape[0]
img_height = x_train.shape[1]
img_width = x_train.shape[2]
num_channels = 1
x_train = x_train.reshape(num_train,1,img_height,img_width)
x_test = x_test.reshape(num_test,1,img_height,img_width)

In [56]:
opt= Adam(learning_rate=0.01)
model.compile(optimizer = opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [57]:
def lr_decay(epoch):
  return 0.01*math.pow(0.666,epoch)
lr_decay_cb = LearningRateScheduler(lr_decay,verbose=True)
model_check_cb = ModelCheckpoint('mnist_dwt.h5',save_best_only=True,monitor='val_loss')

In [58]:
history = model.fit(x_train,y_train_oh,validation_data=(x_test,y_test_oh),epochs=10,batch_size=64,
                    callbacks=[lr_decay_cb,model_check_cb])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
Epoch 1/10
938/938 [==============================] - 9s 9ms/step - loss: 0.2802 - accuracy: 0.9309 - val_loss: 0.0632 - val_accuracy: 0.9800 - lr: 0.0100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.00666.
Epoch 2/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0748 - accuracy: 0.9785 - val_loss: 0.0449 - val_accuracy: 0.9852 - lr: 0.0067

Epoch 00003: LearningRateScheduler reducing learning rate to 0.004435560000000001.
Epoch 3/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0498 - accuracy: 0.9849 - val_loss: 0.0351 - val_accuracy: 0.9878 - lr: 0.0044

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0029540829600000007.
Epoch 4/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0362 - accuracy: 0.9890 - val_loss: 0.0347 - val_accuracy: 0.9887 - lr: 0.0030

Epoch 00005: LearningRateScheduler reducing learning rate to 0.00196741

In [59]:
best_model = tf.keras.models.load_model('mnist_dwt.h5',custom_objects={'DWT_Pooling':DWT_Pooling})

In [60]:
best_model.evaluate(x_test,y_test_oh)
y_preds = np.argmax(m.predict(x_test),axis=1)

313/313 [==============================] - 1s 4ms/step - loss: 0.0245 - accuracy: 0.9931


In [62]:
from sklearn.metrics import classification_report,roc_auc_score
print(classification_report(y_test,y_preds))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       980
           1       0.99      1.00      1.00      1135
           2       1.00      1.00      1.00      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.99      0.99       982
           5       0.99      0.99      0.99       892
           6       1.00      0.99      0.99       958
           7       0.99      1.00      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

